In [1]:
import os
from data_processing import _subset_states, process_smap, process_nldas, process_forcing
from global_paths import project_data_path

### Entire domain

In [4]:
subset_name = "CONUS"
list_of_states = None

In [5]:
# Make directories
output_path = f"{project_data_path}/WBM/precalibration/{subset_name}"

# Main
if not os.path.isdir(output_path):
    os.mkdir(output_path)
    
# Subs
for sub in ["SMAP", "VIC", "NOAH", "MOSAIC"]:
    if not os.path.isdir(f"{output_path}/{sub}"):
        os.mkdir(f"{output_path}/{sub}")

In [6]:
%%time
# SMAP
process_smap(subset_name, list_of_states)

SMAP already processed
CPU times: user 694 µs, sys: 0 ns, total: 694 µs
Wall time: 775 µs


In [7]:
%%time
# NLDAS
process_nldas(subset_name, list_of_states)

CPU times: user 1min 1s, sys: 23.5 s, total: 1min 24s
Wall time: 2min 25s


In [8]:
%%time
# Forcing
process_forcing(subset_name, list_of_states)

CPU times: user 3min 18s, sys: 1min 3s, total: 4min 22s
Wall time: 10min 30s


### Midwest

In [2]:
subset_name = "midwest"
list_of_states = ["North Dakota", "South Dakota", "Nebraska", "Kansas",
                  "Minnesota", "Iowa", "Missouri", 
                  "Wisconsin", "Illinois",
                  "Michigan", "Indiana", "Kentucky", "Ohio"]

In [4]:
# Make directories
output_path = f"{project_data_path}/WBM/calibration/{subset_name}"

# Main
if not os.path.isdir(output_path):
    os.mkdir(output_path)
    
# Subs
for sub in ["SMAP", "VIC", "NOAH", "MOSAIC"]:
    if not os.path.isdir(f"{output_path}/{sub}"):
        os.mkdir(f"{output_path}/{sub}")

In [5]:
%%time
# SMAP
process_smap(subset_name, list_of_states)

CPU times: user 1min 34s, sys: 17.3 s, total: 1min 51s
Wall time: 2min 58s


In [6]:
%%time
# NLDAS
process_nldas(subset_name, list_of_states)

CPU times: user 5min 20s, sys: 26.4 s, total: 5min 46s
Wall time: 9min 23s


In [7]:
%%time
# Forcing
process_forcing(subset_name, list_of_states)

CPU times: user 6min 28s, sys: 32.8 s, total: 7min 1s
Wall time: 10min 3s


### Central US

In [2]:
subset_name = "centralUS"
list_of_states = ['Alabama', 'Arkansas', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Illinois', 
                  'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
                  'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Nebraska', 'New Hampshire', 
                  'New Jersey', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Pennsylvania', 
                  'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Vermont', 'Virginia', 
                  'West Virginia', 'Wisconsin']

In [3]:
# Make directories
output_path = f"{project_data_path}/WBM/calibration/{subset_name}"

# Main
if not os.path.isdir(output_path):
    os.mkdir(output_path)
    
# Subs
for sub in ["SMAP", "VIC", "NOAH", "MOSAIC"]:
    if not os.path.isdir(f"{output_path}/{sub}"):
        os.mkdir(f"{output_path}/{sub}")

In [4]:
%%time
# SMAP
process_smap(subset_name, list_of_states)

SMAP already processed
CPU times: user 131 µs, sys: 817 µs, total: 948 µs
Wall time: 20.2 ms


In [5]:
%%time
# NLDAS
process_nldas(subset_name, list_of_states)

CPU times: user 838 µs, sys: 103 µs, total: 941 µs
Wall time: 13.7 ms


In [6]:
%%time
# Forcing
process_forcing(subset_name, list_of_states)

CPU times: user 1min 38s, sys: 7.48 s, total: 1min 46s
Wall time: 2min 15s
